In [2]:
!pip install fairseq --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 57.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11288625 sha256=dc867a198293c442afc2fde1d5315fe26c862ea5cea1270643589a2b4a9f143e
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
Successfully built fairseq


In [3]:
!pip install pip==23.1.2

In [5]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141214 sha256=65b569ac7d483b0b002e3c4abf38aa8ba34c728cc27d39efd963033846b430c4
  Stored in directory: /root/.cache/pip/wheels/a7/20/bd/e1477d664f22d99989fd28ee1a43d6633dddb5cb9e801350d5
Successfully built antlr4-python3-runtime


In [7]:
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [8]:
!pip install sentencepiece

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
train_src = '/content/drive/MyDrive/Code_transfer/train.src'
train_tgt = '/content/drive/MyDrive/Code_transfer/train.tgt'
valid_src = '/content/drive/MyDrive/Code_transfer/valid.src'
valid_tgt = '/content/drive/MyDrive/Code_transfer/valid.tgt'
test_src = '/content/drive/MyDrive/Code_transfer/test.src'
test_tgt = '/content/drive/MyDrive/Code_transfer/test.tgt'

In [11]:
# Function to read and process the dataset
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # Split lines into tokens
    tokenized_lines = [line.split() for line in lines]
    return tokenized_lines

# Function to calculate vocabulary size and max length
def calculate_vocab_size_and_max_length(tokenized_lines):
    vocab = set()  # To store unique words
    max_length = 0  # To track maximum sequence length

    for tokens in tokenized_lines:
        vocab.update(tokens)  # Add tokens to vocab set
        max_length = max(max_length, len(tokens))  # Update max length

    return len(vocab), max_length

# Process each dataset and calculate vocabulary size and max sequence length
datasets = {
    'train_src': train_src,
    'train_tgt': train_tgt,
    'valid_src': valid_src,
    'valid_tgt': valid_tgt,
    'test_src': test_src,
    'test_tgt': test_tgt
}

for dataset_name, dataset_path in datasets.items():
    tokenized_lines = read_file(dataset_path)
    vocab_size, max_length = calculate_vocab_size_and_max_length(tokenized_lines)
    print(f"Dataset: {dataset_name}")
    print(f"  Vocabulary Size: {vocab_size}")
    print(f"  Max Sequence Length: {max_length}\n")


Dataset: train_src
  Vocabulary Size: 63251
  Max Sequence Length: 42105

Dataset: train_tgt
  Vocabulary Size: 1533
  Max Sequence Length: 573

Dataset: valid_src
  Vocabulary Size: 39242
  Max Sequence Length: 42105

Dataset: valid_tgt
  Vocabulary Size: 744
  Max Sequence Length: 564

Dataset: test_src
  Vocabulary Size: 497
  Max Sequence Length: 1432

Dataset: test_tgt
  Vocabulary Size: 62
  Max Sequence Length: 138



## Preprocess the Dataset

In [12]:
import sentencepiece as spm

# Function to count the number of unique tokens in a dataset
def get_vocab_size(input_file):
    vocab = set()
    with open(input_file, 'r') as f:
        for line in f:
            tokens = line.split()
            vocab.update(tokens)
    return len(vocab)

# Get vocab size for train_src and train_tgt
train_src_vocab_size = get_vocab_size(train_src)
train_tgt_vocab_size = get_vocab_size(train_tgt)

# Train BPE models for source and target datasets
spm.SentencePieceTrainer.train(input=train_src, model_prefix='bpe_src', vocab_size=470)
spm.SentencePieceTrainer.train(input=train_tgt, model_prefix='bpe_tgt', vocab_size=470)

# Apply BPE encoding to the datasets
def apply_bpe(input_file, output_file, model_file):
    sp = spm.SentencePieceProcessor(model_file=model_file)
    with open(output_file, 'w') as out_file:
        with open(input_file) as in_file:
            for line in in_file:
                out_file.write(' '.join(sp.encode(line, out_type=str)) + '\n')

# Apply BPE to source and target files
apply_bpe(train_src, 'train_bpe.src', 'bpe_src.model')
apply_bpe(train_tgt, 'train_bpe.tgt', 'bpe_tgt.model')
apply_bpe(valid_src, 'valid_bpe.src', 'bpe_src.model')
apply_bpe(valid_tgt, 'valid_bpe.tgt', 'bpe_tgt.model')
apply_bpe(test_src, 'test_bpe.src', 'bpe_src.model')
apply_bpe(test_tgt, 'test_bpe.tgt', 'bpe_tgt.model')


In [13]:
def wordwise_encoding(input_file, output_file):
    with open(input_file, 'r') as in_file, open(output_file, 'w') as out_file:
        for line in in_file:
            out_file.write(' '.join(line.strip().split()) + '\n')

# Apply word-wise encoding to source and target files
wordwise_encoding(train_src, 'train_wordwise.src')
wordwise_encoding(train_tgt, 'train_wordwise.tgt')
wordwise_encoding(valid_src, 'valid_wordwise.src')
wordwise_encoding(valid_tgt, 'valid_wordwise.tgt')
wordwise_encoding(test_src, 'test_wordwise.src')
wordwise_encoding(test_tgt, 'test_wordwise.tgt')


# Preprocess Data with Fairseq

In [14]:
# BPE data preprocessing
!fairseq-preprocess --source-lang src --target-lang tgt \
    --trainpref train_bpe --validpref valid_bpe --testpref test_bpe \
    --destdir data-bin/bpe --workers 20

# Word-wise data preprocessing
!fairseq-preprocess --source-lang src --target-lang tgt \
    --trainpref train_wordwise --validpref valid_wordwise --testpref test_wordwise \
    --destdir data-bin/wordwise --workers 20


2024-10-23 15:54:22 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-10-23 15:54:23 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None

In [15]:
# positional data preprocessing

!fairseq-preprocess --source-lang src --target-lang tgt \
    --trainpref train_wordwise --validpref valid_wordwise --testpref test_wordwise \
    --destdir data-bin/positional --workers 20

2024-10-23 15:56:14 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-10-23 15:56:14 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None

# Train the Models
 BPE model training

In [17]:
!fairseq-train data-bin/bpe \
    --arch transformer --share-decoder-input-output-embed \
    --encoder-layers 6 --decoder-layers 6 \
    --max-tokens 50000 --lr 0.01 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.1 --dropout 0.1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 50 --save-dir checkpoints/bpe_transformer \
    --max-source-positions 42105 --max-target-positions 573 \
    --skip-invalid-size-inputs-valid-test

Streaming output truncated to the last 5000 lines.
|---------------------------------------------------------------------------|
| GPU reserved segments |      56    |      75    |     139    |      83    |
|       from large pool |      15    |      33    |      84    |      69    |
|       from small pool |      41    |      43    |      55    |      14    |
|---------------------------------------------------------------------------|
| Non-releasable allocs |      10    |      22    |     998    |     988    |
|       from large pool |       8    |      19    |     604    |     596    |
|       from small pool |       2    |       5    |     394    |     392    |
|---------------------------------------------------------------------------|
| Oversize allocations  |       0    |       0    |       0    |       0    |
|---------------------------------------------------------------------------|
| Oversize GPU segments |       0    |       0    |       0    |       0    |
|============

## Word-wise model training

In [ ]:
!fairseq-train data-bin/wordwise \
    --arch transformer --share-decoder-input-output-embed \
    --encoder-layers 6 --decoder-layers 6 \
    --max-tokens 50000 --lr 0.0005 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.1 --dropout 0.1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 4 --save-dir checkpoints/wordwise_transformer\
    --max-source-positions 42105 --max-target-positions 573 \
    --skip-invalid-size-inputs-valid-test

2024-10-21 16:01:08.200901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 16:01:08.233995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 16:01:08.253995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 16:01:08.278674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 16:01:09.893336: W tensorflow/comp

# positional model training

In [ ]:
!fairseq-train data-bin/positional \
    --arch transformer --share-decoder-input-output-embed \
    --encoder-layers 6 --decoder-layers 6 \
    --max-tokens 50000 --lr 0.0005 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.1 --dropout 0.1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 4 --save-dir checkpoints/positional_transformer\
    --max-source-positions 42105 --max-target-positions 573 \
    --skip-invalid-size-inputs-valid-test

2024-10-21 16:48:59.956321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 16:48:59.990349: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 16:49:00.000330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 16:49:00.022832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 16:49:01.726510: W tensorflow/comp

# Evaluate the Models

In [ ]:
# BPE model evaluation
!fairseq-generate data-bin/bpe --path checkpoints/bpe_transformer/checkpoint_best.pt \
    --beam 5 --remove-bpe --max-source-positions 42105 --max-target-positions 500 \
    --skip-invalid-size-inputs-valid-test > bpe_output.txt



2024-10-21 18:21:18.006477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 18:21:18.037730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 18:21:18.044325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 18:21:18.059462: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 18:21:19.181380: W tensorflow/comp

In [ ]:
# Word-wise model evaluation
!fairseq-generate data-bin/wordwise --path checkpoints/wordwise_transformer/checkpoint_best.pt --beam 5 > wordwise_output.txt

# Positional encoding (is included by default in Transformer models)
!fairseq-generate data-bin/positional --path checkpoints/positional_transformer/checkpoint_best.pt --beam 5 > positional_output.txt
